<a href="https://colab.research.google.com/github/stevecwb/Modelando-a-epidemia-de-Covid-19/blob/main/Modelando_a_epidemia_de_Covid_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo SEIR

O modelo SEIR consiste em resolver as seguintes equações diferenciais:
$$\begin{aligned}
\frac{d \mathcal{S}}{d t} &=-\frac{r_{0}}{T_{\mathrm{inf}}} \mathcal{S} \mathcal{I} \\
\frac{d \mathcal{E}}{d t} &=\frac{r_{0}}{T_{\mathrm{inf}}} \mathcal{S} \mathcal{I}-\frac{1}{T_{\mathrm{inc}}} \mathcal{E} \\
\frac{d \mathcal{I}}{d t} &=\frac{1}{T_{\mathrm{inc}}} \mathcal{E}-\frac{1}{T_{\mathrm{inf}}} \mathcal{I} \\
\frac{d \mathcal{R}}{d t} &=\frac{1}{T_{\mathrm{inf}}} \mathcal{I}
\end{aligned}$$

Sabendo que, segundo os dados do enunciado, tem-se os seguintes valores:
- Número de reprodução basal: $r_0=2,5$
- Tempos médio de incubação e infecção: $T_{\mathrm{inc}}=5,2\text{ dias}$ e $T_{\mathrm{inf}}=2,9\text{ dias}$


O primeiro passo é compilar **todas** as células. Após isso, basta alterar os inputs seguindo as instruções de cada um:


*   ```Default```: se estiver **marcado**, vai escolher os **valores padrão** escritos no enunciado do problema. Mesmo se as outras variáveis estiverem diferentes, os gráficos gerados serão os padrões.

*   ```r_0```: slider para variar os valores da taxa de reprodução basal ($r_0$). Só funciona se a opção ```Default``` estiver **desmarcada**. **Importante**: se ```Default``` estiver **desmarcado**, mas se ```Random``` estiver **marcado**, então a taxa vai ser aleatória e não vai ser o valor do slider.

*   ```Random```: se ```Default``` estiver  **desmarcado** e se ```Random``` estiver **marcado**, então a taxa de reprodução basal ($r_0$) será **aleatória** (para mais detalhes, clique neste [link](https://colab.research.google.com/drive/1N8DGB3jnjGZZVQX8LPEb0vx-rFFyoVIL#scrollTo=sziKWQJ3WrA5&line=1&uniqifier=1)). Esta opção é interessante para simular uma variação na taxa, além de conseguir oscilações nos valores de forma a simular uma possível segunda onda da doença.

* ```T_inc```: slider para variar os valores do tempo médio de incubação ($T_{\mathrm{inc}}$). Só funciona se a opção ```Default``` estiver **desmarcada**.

* ```T_inf```: slider para variar os valores do tempo médio de infecção ($T_{\mathrm{inf}}$). Só funciona se a opção ```Default``` estiver **desmarcada**.


In [40]:
#@title Escolha os valores iniciais { run: "auto", display-mode: "form" }
Default = False #@param {type:"boolean"}
r_0 = 7.2 #@param {type:"slider", min:0.1, max:10, step:0.05,value:2.5}
Random = True #@param {type:"boolean"}
T_inc = 5 #@param {type:"slider", min:0.1, max:10, step:0.1}
T_inf = 2.8 #@param {type:"slider", min:0.1, max:10, step:0.1}





if Default:
  r_0=2.5
  T_inc=5.2
  T_inf=2.9
import ipywidgets as widgets
import plotly.graph_objects as go
from IPython.display import display
from IPython.display import HTML
import numpy as np
#%matplotlib widget
#%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set();


#r_0=slider.value #número de reprodução basal
#T_inc=slider1.value #tempo de incubação
#T_inf=slider2.value #tempo de infecção
tf=498 #número final de dias da simulação
dt=1 # passo (1 dia)
t=np.arange(0,tf+0.5,dt) # vetor com todos os dias até o 498º dia
n=len(t) #tamanho do vetor t => número de dias da simulação

S = np.ones(n)*0.999999
E = np.ones(n)*(10**-6)
I = np.zeros(n)
R = np.zeros(n)

def f(y):
    a = (r_0/T_inf)*y[0]*y[2]
    b = (1/T_inc)*y[1]
    c = (1/T_inf)*y[2]
    S_ = -a
    E_ = (a - b)
    I_ = (b - c)
    R_ = c
    return [S_,E_,I_,R_]

for i in range(1,n):
    if Random == True and Default == False:
      a=np.random.rand(1)[0]*5
      r_0=abs(a*np.cos(2.5*np.pi*t[i]/498))+0.5 
    y05 = f([S[i-1],E[i-1],I[i-1],R[i-2]])
    S_05 = S[i-1] + y05[0]*(dt/2)
    E_05 = E[i-1] + y05[1]*(dt/2)
    I_05 = I[i-1] + y05[2]*(dt/2)
    R_05 = R[i-1] + y05[3]*(dt/2)
    y_ = [S_05,E_05,I_05,R_05]
    S[i] = S[i-1] + dt*f(y_)[0]
    E[i] = E[i-1] + dt*f(y_)[1]
    I[i] = I[i-1] + dt*f(y_)[2]
    R[i] = R[i-1] + dt*f(y_)[3]


# Layout
layout = go.Layout(
    title="Proporção do modelo SEIR versus tempo",
    title_x=0.5,
    xaxis=dict(
        title="Tempo (dias)"
        ),
        yaxis=dict(
        title="Porcentagem da População (%)"
        ) 
)
fig = go.Figure(layout = layout)

# Add scatter trace with medium sized markers
fig.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=S,
        name='Suscetíveis',
        marker=dict(
            color='darkturquoise',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=E,
        name='Expostos',
        marker=dict(
            color='darkmagenta',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=I,
        name='Infectados',
        marker=dict(
            color='red',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=R,
        name='Recuperados',
        marker=dict(
            color='yellow',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)

S0 = np.ones(n)
E0 = np.ones(n)
I0 = np.zeros(n)
R0 = np.zeros(n)
L0 = np.zeros(n)
P0 = np.zeros(n)
W0 = np.zeros(n)

for j in range(n):
    S0[j] = int(S[j]*44040000)
    E0[j] = int(E[j]*44040000)
    I0[j] = int(I[j]*44040000)
    R0[j] = int(R[j]*44040000)
    L0[j] = int((27*44040000)/100000)
    if j>=1:
        P0[j] = int(abs(R0[j-1]-R0[j])*0.03)
    
for p in range(n):
    if p==0:
        W0[p]=P0[p]
    elif p==1:
        W0[p]=P0[p]+P0[p-1]
    elif p==2:
        W0[p]=P0[p]+P0[p-1]+P0[p-2]
    elif p==3:
        W0[p]=P0[p]+P0[p-1]+P0[p-2]+P0[p-3]
    elif p==4:
        W0[p]=P0[p]+P0[p-1]+P0[p-2]+P0[p-3]+P0[p-4]
    elif p==5:
        W0[p]=P0[p]+P0[p-1]+P0[p-2]+P0[p-3]+P0[p-4]+P0[p-5]
    else:
        W0[p]=P0[p]+P0[p-1]+P0[p-2]+P0[p-3]+P0[p-4]+P0[p-5]+P0[p-6]

pessoas_sem_uti = []
for b in range(len(L0)):
    if W0[b]>=L0[b]:
        pessoas_sem_uti.append(W0[b]-L0[b])

layout1 = go.Layout(
    title="População de SP versus tempo",
    title_x=0.5,
    xaxis=dict(
        title="Tempo (dias)"
        ),
        yaxis=dict(
        title="População"
        ),
    annotations=[
            go.layout.Annotation(
                text=f"O número aproximado<br> de pessoas que ficarão<br> sem UTI é <b>{sum(pessoas_sem_uti):,.0f}<b>",
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                x=1.2,
                y=0,
                bordercolor='rgb(42, 63, 126)',
                borderwidth=1
            )
        ]
)


fig1 = go.Figure(layout = layout1)

fig1.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=S0,
        name='Suscetíveis',
        marker=dict(
            color='darkturquoise',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig1.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=E0,
        name='Expostos',
        marker=dict(
            color='darkmagenta',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig1.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=I0,
        name='Infectados',
        marker=dict(
            color='red',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig1.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=R0,
        name='Recuperados',
        marker=dict(
            color='yellow',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig1.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=L0,
        name='Leitos',
        marker=dict(
            color='orange',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig1.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=W0,
        name='Necessidade de Leitos',
        marker=dict(
            color='green',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
 
display(fig, fig1)

